In [ ]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

#import pingouin as pg

In [ ]:
sys.path.insert(0,"/home/container/notebooks/missions-cways/mobivia-revenue-forecast/mobivia_revenue_forecast/src")

#from random_draw_selection_feature import random_draw_selection_feature
from get_score_params_predict import get_score_params_predict
from evaluate_prediction import evaluate_prediction
from evaluate_prediction import evaluate_one_leave_out
from get_clean_feature_and_ca import get_clean_feature_and_ca
from display_model import display_model
from display_curve import display_curve
from exhaustive_selection_feature import exhaustive_selection_feature

# Constantes

In [ ]:
predict_year_list=[2021, 2022]
train_year_list=[2000, 2001,2002,2003,2004,2005,2006,2007,2008,2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,2017,2018, 2019, 2020]
to_keep_index_list=[k for k in range(2000, 2023)]#pour l'étape de pré-traitement : toutes les lignes dont on aura besoin
variable_number=3
#draw_nb=10
nb_model_to_select=100
growth_rate_bool=False #True pour avoir le modèle en taux de croissance
ca_col_str="NOR_CA_TOT_ACTU"

GLOBAL_MARKET_VAR_LIST=[
    "REV_DISPO_BRUT",
    "PA_REV_DISPO_BRUT",
    "PA_MENAGE", 
    "IND_PRIX_CONSO",
    "CONSO_FINALE_MENAGES",
    "PIB",
    "CHOMAGE",
    "TX_EPARGNE",
    "CONF_MENAGES",
    "ICA",
    "CA_E_COMMER_INIT"
]
"""AUTO_MARKET_VAR_LIST=[
    "NORAUTO_INVEST_PUB",
    "VOL_PARC",
    "VP03",
    "VP45",
    "VP67",
    "VP810",
    "VP1115",
    "VP610",
    "TXVP03",
    "TXVP45",
    "TXVP67",
    "TXVP810",
    "TXVP1115",
    "TXVP610",
    "IMMAT_VN",
    "IMMAT_VO"
]"""
AUTO_MARKET_VAR_LIST=[
    "TXVP610",
    "VP610"
]
MOBILITY_VAR_LIST=[
    "MODAL_SHARE_VP",
    "MODAL_SHARE_RAIL"
    "VP_KM_ANNUEL",
    "CONSO_CARBURANT",
    "FREQ_CENTRE_COMMER",
    "IND_PRIX_UTILISATION_VEHICULE",
    "PRIX_GASOIL",
    "ECART_TEMPERATURE",
]

#enlever l'année 2000 de l'entrainement si growth rate bool est Vrai

# Import et pré-traitement

In [ ]:
file_path = "/home/container/notebooks/bases/missions-cways/mobivia/mobivia_revenue_forecast/inputs/Consolidation_data_all.xlsx"

In [ ]:
france_pdf = pd.read_excel(open(file_path, 'rb'),sheet_name='France',skiprows=5, index_col = 0)

In [ ]:
feature_pdf, ca_pdf=get_clean_feature_and_ca(france_pdf, ca_col_str, to_keep_index_list, growth_rate_bool)

In [ ]:
print(feature_pdf, ca_pdf)

# Visualisation valeurs aberrantes et manquantes

In [ ]:
feature_drop_pdf=feature_pdf.drop(['ETAT_CRISE','ECART_TEMPERATURE','CA_E_COMMER_INIT'], axis=1)
lgt=len(feature_drop_pdf.columns)
print(feature_drop_pdf.columns, lgt)
feature_minus_mean_pdf = feature_drop_pdf.apply(lambda x: x/x.mean(), axis=0)

rename_list=[k for k in range(lgt)]
k=0
for col in feature_drop_pdf.columns:
    feature_minus_mean_pdf.rename(columns = {col: rename_list[k]}, inplace=True)
    k+=1
bxplot=feature_minus_mean_pdf.boxplot()

# vérification hypothèses normalité

In [ ]:
feature_pdf=feature_drop_pdf

In [ ]:
from scipy import stats
#kolmogorov
#stats.kstest(feature_pdf, "norm")
#shapiro
feature_pdf.apply(stats.shapiro)

# vérification hypothèses homoscédasticité

In [ ]:
for col in feature_pdf.columns:
    stats.levene(feature_pdf[col].to_numpy())

In [ ]:
stats.levene(feature_pdf)

In [ ]:
for col in feature_pdf.columns:
    print(feature_pdf[col].to_numpy())

# Sélection des meilleures variables

In [ ]:
best_model_pdf =(
    exhaustive_selection_feature(
        feature_pdf, 
        ca_pdf,
        train_year_list,
        predict_year_list, 
        variable_number, 
        nb_model_to_select, 
        GLOBAL_MARKET_VAR_LIST, 
        AUTO_MARKET_VAR_LIST,
        MOBILITY_VAR_LIST
    )
)
best_model_pdf = best_model_pdf.reset_index(drop=True)

# affichage

In [ ]:
display_pdf = display_model(best_model_pdf, variable_number)

In [ ]:
display_pdf

In [ ]:
display_pdf.to_excel("/home/container/notebooks/bases/missions-cways/mobivia/mobivia_revenue_forecast/outputs/norauto_best_models.xlsx")

# Tracé de la courbe

In [ ]:
for k in range(nb_model_to_select):
    print("numéro modèle : "+str(k))
    chosen_cols=list(list(best_model_pdf["params_list"])[k].keys())
    param_dict, score, estimated_serie = get_score_params_predict(feature_pdf[chosen_cols], ca_pdf, [2021, 2022], False)
    compare_results_df = pd.DataFrame(
        list(zip(ca_pdf.tolist(), estimated_serie)),
        columns = ['real_values', 'estimated_values'],
        index = ca_pdf.index
    )
    plt.figure(figsize=(12,8))
    plt.plot(compare_results_df)
    plt.show()
    

# choix du modèle

In [ ]:
numero_choisi = 1 #clef du modèle choisi deux cases au dessus
chosen_cols=list(list(best_model_pdf["params_list"])[numero_choisi].keys())
print(chosen_cols)
#chosen_cols=["CONSO_FINALE_MENAGES", "TXVP03", "VP45"]

# Prédiction, score, paramètres

In [ ]:
param_dict, score, estimated_serie = get_score_params_predict(feature_pdf[chosen_cols], ca_pdf, [2021, 2022], intercept_bool=True)

In [ ]:
print("score", score)
print("params_dict", param_dict)

In [ ]:
param_df = pd.DataFrame.from_dict(param_dict, orient='index')
param_df.to_excel("/home/container/notebooks/bases/missions-cways/mobivia/mobivia_revenue_forecast/outputs/params_ca_norauto_modele"+str(numero_choisi)+".xlsx")
param_df.head()

In [ ]:
compare_results_df = pd.DataFrame(
    list(zip(ca_pdf.tolist(), estimated_serie)),
    columns = ['real_values', 'estimated_values'],
    index = ca_pdf.index
)

compare_results_df.to_excel("/home/container/notebooks/bases/missions-cways/mobivia/mobivia_revenue_forecast/outputs/prev_ca_norauto_modele"+str(numero_choisi)+".xlsx")
compare_results_df.head(n=50)

In [ ]:
plt.figure(figsize=(12,8))
#compare_results_df.drop([2021, 2022], axis=0).plot()
compare_results_df.plot()
plt.xlabel("Année")
plt.ylabel("Chiffre d'affaires")
plt.show()

In [ ]:
compare_results_df=display_curve(compare_results_df, train_year_list, growth_rate_bool)
compare_results_df.to_excel("/home/container/notebooks/bases/missions-cways/mobivia/mobivia_revenue_forecast/outputs/orev_ca_norauto.xlsx")
#résidus négatifs : estimated_value supérieure à real_value
#exprimés en % d'écart => fonctionne mal avec les taux de croissances
#pour le taux de croissance, il faudrait juste une différence, pas relative

In [ ]:
prediction_score_dict = evaluate_prediction(compare_results_df,predict_year_list)

In [ ]:
print(prediction_score_dict)

In [ ]:
#le one leave out permet de tester le sur-apprentissage
one_leave_out_score_dict, score_moyen = evaluate_one_leave_out(feature_pdf[chosen_cols], ca_pdf, 2000, 2020)

In [ ]:
print(pd.DataFrame.from_dict(one_leave_out_score_dict, orient='index'), "score_moyen", score_moyen)